In [ ]:
pip install pandas


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.1.0-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


In [50]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [22]:
url = "https://en.wikipedia.org/wiki/breast_cancer"

In [23]:
raw = requests.get(url)

In [24]:
raw

<Response [200]>

In [25]:
soup = BeautifulSoup(raw.text)

In [26]:
#print(soup)

In [27]:
failed_retrievals = {}
def GetData(url):
    try:
        print(f"Fetching data from {url}")
        raw = requests.get(url)
        soup = BeautifulSoup(raw.text, 'html.parser')
        table = soup.find('table', {'class': 'infobox ib-medical-condition'})
        data = {}
        requirements = {'Symptoms', 'Risk factors', 'Treatment', 'Frequency', 'Diagnostic method', 'Causes'}
        if table:
            rows = table.find_all('tr')
            for row in rows:
                headings = row.find_all('th')
                details = row.find_all('td')
                if headings and details:
                    for heading, detail in zip(headings, details):
                        if heading.text.strip() in requirements:
                            data[heading.text.strip()] = detail.text.strip()
        else:
            print(f"No infobox found for {url}")
            failed_retrievals[url] = "No infobox found"
        return data
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        failed_retrievals[url] = str(e)
        return None


In [28]:
temp = GetData("https://en.wikipedia.org/wiki/breast_cancer")

Fetching data from https://en.wikipedia.org/wiki/breast_cancer


In [29]:
print(len(temp))

5


In [30]:
base_url = "https://en.wikipedia.org/wiki/"
total = {}

In [36]:
file = pd.read_csv('data.csv')
file[:5]

,Value
0,injury to the hip
1,peyronie disease
2,cornea infection
3,ear drum damage
4,brain cancer


In [41]:
diseases = file['Value']

In [42]:
for d in diseases:
    lists = d.split()
    dis = '_'.join(lists)
    url = base_url + dis
    data = GetData(url)
    if data:
        total[dis] = data
        print(f"Data for {dis}: {data}")

Fetching data from https://en.wikipedia.org/wiki/injury_to_the_hip
No infobox found for https://en.wikipedia.org/wiki/injury_to_the_hip
Fetching data from https://en.wikipedia.org/wiki/peyronie_disease
Data for peyronie_disease: {'Causes': 'Unknown[2]', 'Frequency': '~10% of men[2]'}
Fetching data from https://en.wikipedia.org/wiki/cornea_infection
No infobox found for https://en.wikipedia.org/wiki/cornea_infection
Fetching data from https://en.wikipedia.org/wiki/ear_drum_damage
No infobox found for https://en.wikipedia.org/wiki/ear_drum_damage
Fetching data from https://en.wikipedia.org/wiki/brain_cancer
Data for brain_cancer: {'Symptoms': 'Vary depending on the part of the brain involved, headaches, seizures, problem with vision, vomiting, mental changes[1][2]', 'Causes': 'Usually unknown[2]', 'Risk factors': 'Neurofibromatosis, exposure to vinyl chloride, Epstein–Barr virus, ionizing radiation[1][2][3]', 'Diagnostic method': 'Computed tomography, magnetic resonance imaging, tissue b

In [43]:
total

{'peyronie_disease': {'Causes': 'Unknown[2]', 'Frequency': '~10% of men[2]'},
 'brain_cancer': {'Symptoms': 'Vary depending on the part of the brain involved, headaches, seizures, problem with vision, vomiting, mental changes[1][2]',
  'Causes': 'Usually unknown[2]',
  'Risk factors': 'Neurofibromatosis, exposure to vinyl chloride, Epstein–Barr virus, ionizing radiation[1][2][3]',
  'Diagnostic method': 'Computed tomography, magnetic resonance imaging, tissue biopsy[1][2]',
  'Treatment': 'Surgery, radiation therapy, chemotherapy[1]',
  'Frequency': '1.2 million nervous system cancers (2015)[5]'},
 'dumping_syndrome': {'Causes': 'Gastrectomy, gastric bypass surgery, diabetes, esophageal surgery, absent or inefficient pyloric sphincter, pyloric stenosis'},
 'gallstone': {'Symptoms': 'None, crampy pain in the right upper abdomen[2][3][4]',
  'Risk factors': 'Birth control pills, pregnancy, family history, obesity, diabetes, liver disease, rapid weight loss[2]',
  'Diagnostic method': 'Ba

In [56]:
len(total)

379

In [19]:
delete = [k for k, v in total.items() if not v]
for k in delete:
    del data[k]

In [57]:
len(total)

379

In [21]:
failed_retrievals

{'https://en.wikipedia.org/wiki/Knee_Pain': 'No infobox found',
 'https://en.wikipedia.org/wiki/chronic_knee_pain': 'No infobox found',
 'https://en.wikipedia.org/wiki/Food_poisoning': 'No infobox found',
 'https://en.wikipedia.org/wiki/trichomonas_infection': 'No infobox found',
 'https://en.wikipedia.org/wiki/Urinary_incontinence_in_women': 'No infobox found',
 'https://en.wikipedia.org/wiki/Atopic_Dermatitis_(Eczema)': 'No infobox found',
 'https://en.wikipedia.org/wiki/open_wound_of_the_jaw': 'No infobox found',
 'https://en.wikipedia.org/wiki/Alopecia_Areata': 'No infobox found',
 'https://en.wikipedia.org/wiki/developmental_disability': 'No infobox found',
 'https://en.wikipedia.org/wiki/cellulitis_or_abscess_of_mouth': 'No infobox found',
 'https://en.wikipedia.org/wiki/heart_block': 'No infobox found',
 'https://en.wikipedia.org/wiki/Coronavirus_(COVID-19)': 'No infobox found',
 'https://en.wikipedia.org/wiki/Polycystic_ovary_syndrome_(PCOS)': 'No infobox found',
 'https://en.w

In [22]:
import csv
import numpy as np

In [42]:
all_keys = set()
for data in total.values():
    all_keys.update(data.keys())

fieldnames = ['Disease'] + list(all_keys)
with open('output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for disease, data in total.items():
        row = {'Disease': disease}
        row.update({key: data.get(key, np.nan) for key in all_keys})
        writer.writerow(row)


In [52]:
dataset = pd.read_csv('bigdata.csv')

In [53]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Disease            843 non-null    object
 1   Risk factors       451 non-null    object
 2   Diagnostic method  680 non-null    object
 3   Treatment          670 non-null    object
 4   Causes             629 non-null    object
 5   Symptoms           777 non-null    object
 6   Frequency          691 non-null    object
dtypes: object(7)
memory usage: 46.2+ KB


In [54]:
dataset.head()

,Disease,Risk factors,Diagnostic method,Treatment,Causes,Symptoms,Frequency
0,ovarian_cyst,NaN,Ultrasound[1],"Conservative management, pain medication, surg...",NaN,"None, bloating, lower abdominal pain, lower ba...",8% symptomatic before menopause[1]
1,Rheumatoid_Arthritis,NaN,"Based on symptoms, medical imaging, blood test...",NaN,Unknown[1],"Warm, swollen, painful joints[1]",0.5–1% (adults in developed world)[3]
2,fibromyalgia,NaN,Based on symptoms after ruling out other poten...,Sufficient sleep and exercise[5],Unknown[4][5],"Widespread pain, feeling tired, sleep problems...",2%[4]
3,rotator_cuff_injury,NaN,"Based on symptoms, examination, medical imagin...","Pain medication, specific exercises, surgery[1]",NaN,"Shoulder pain, weakness[1]",Common[2]
4,rheumatoid_arthritis,NaN,"Based on symptoms, medical imaging, blood test...",NaN,Unknown[1],"Warm, swollen, painful joints[1]",0.5–1% (adults in developed world)[3]


In [55]:

df = pd.read_csv('output.csv')

# Count NaNs in each column
nan_counts = df.isna().sum()

# Print the number of NaNs for each column
print("Number of NaN values per column:")
print(nan_counts)


FileNotFoundError: [Errno 2] No such file or directory: 'output.csv'

In [ ]:
df1[:2]

,0
0,Value
1,intestinal obstruction


In [ ]:
df2[0:2]

,0
0,Value
1,Rheumatoid Arthritis


In [ ]:
p[:2]

0                     Value
1    intestinal obstruction
Name: 0, dtype: object

In [ ]:
g[:2]

0                   Value
1    Rheumatoid Arthritis
Name: 0, dtype: object

In [ ]:
len(data)

1593

In [3]:
original = pd.read_csv('data.csv')

In [4]:
founded = pd.read_csv('output.csv')

In [11]:
founded_arr = founded['Disease'].values
founded_arr[:2]

array(['peyronie_disease', 'brain_cancer'], dtype=object)

In [16]:
original_arr = original['Value'].values
original_arr[:2]

array(['injury to the hip', 'peyronie disease'], dtype=object)

In [17]:
not_founded = []
for i in original_arr:
    if i not in founded_arr:
        not_founded.append(i)

In [45]:
df = pd.read_csv('output_file.csv')

In [47]:
y = df['Disease']
y

0            peyronie_disease
1                brain_cancer
2            dumping_syndrome
3                   gallstone
4           alzheimer_disease
                ...          
375               spondylitis
376               pharyngitis
377          thrombocytopenia
378    carpal_tunnel_syndrome
379          abdominal_hernia
Name: Disease, Length: 380, dtype: object